In [1]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit

In [2]:
# plotting the kinetic curves of the RAFT polymerization
# conversion over time

# get all conversion values
conversion_list = []
for column in srt.df.columns:
    if "yield" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print (hours_list)

['t0h-yield', 't1h-yield', 't2h-yield', 't4h-yield', 't8h-yield', 't15h-yield']
[0, 1, 2, 4, 8, 15]


In [3]:
# plotting the kinetic curves of the RAFT polymerization
# create a dataframe to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    kinetic_curve_entries = pd.DataFrame({"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list})
    kinetic_curve_entries["experiment"] = polymerisation_kinetic["Experiment number"]+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)
    # fig = px.line(x=hours_list, y=yields, title=f"Kinetic curve of {polymerisation_kinetic['Experiment number']} polymerization")
    # fig.show()


In [48]:
px.line(kinetic_curves_df[:], x="time", y="conversion", color="experiment")

In [43]:
# and fit curve and the derivation
# and find the maximum of the derivation
# intersect/superimpose all curves

'''    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

# ToDo: check if this is the correct derivative
def sigmoid_derivative(x, L ,x0, k, b):
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y

fig = px.line(title="kinetic curve fit", labels={"x":"time", "y":"conversion"})
for kinetic_curve in kinetic_curves:
    ydata = kinetic_curve["conversion"].values.astype(float)
    xdata = kinetic_curve["time"].values
    p0 = [max(ydata), 1] # this is a mandatory initial guess
    notnan = ~(np.isnan(xdata) | np.isnan(ydata))
    xdata = xdata[notnan]
    ydata = ydata[notnan]

    popt, pcov = curve_fit(neg_growth, xdata, ydata, p0, maxfev=800*10)
    print(popt)

    # plot the fit and data
    extended_xdata = np.linspace(min(xdata)-2, max(xdata)+2, 100)
    y_extended = neg_growth(extended_xdata, *popt)
    notnan_extended = ~(np.isnan(extended_xdata) | np.isnan(y_extended))
    extended_xdata = extended_xdata[notnan_extended]
    y_extended = y_extended[notnan_extended]
#    extended_xdata = np.array(list(range(min(xdata)-2, min(xdata)-1, 1)) + xdata + list(range(max(xdata)+1, max(xdata)+3, 1)))
    fig.add_scatter(x=xdata, y=ydata, mode="lines", opacity=0.5)
    fig.add_scatter(x=extended_xdata, y=neg_growth(extended_xdata, *popt), mode="lines", opacity=0.5)
    # fig.add_scatter(x=extended_xdata, y=neg_growth_derivative(extended_xdata, *popt), mode="lines", opacity=0.5)
    # print(neg_growth(extended_xdata, *popt))
    # break
fig.show()



[0.73294429 0.07987728]
[ 0  1  2  4  8 15] [0.         0.08697079 0.10436495 0.20577617 0.33212996 0.51657368]
[0.80099289 0.13403323]
[ 0  1  2  4  8 15] [0.         0.12338736 0.18921601 0.32782005 0.52034403 0.69732054]
[0.98498259 0.02649672]
[ 0  1  2  4  8 15] [0.         0.03891051 0.04571984 0.09824903 0.18774319 0.32328145]
[0.92499421 0.06329434]
[ 0  1  2  4  8 15] [0.         0.06908095 0.0940353  0.20572124 0.37401096 0.56512477]
[1.05269192 0.09152414]
[ 0  1  2  8 15] [0.         0.11448317 0.18088942 0.53455529 0.7905649 ]
[4.74356588 0.00478091]
[ 0  1  2  4  8 15] [0.         0.05303495 0.05211527 0.07541386 0.17841815 0.3289393 ]
[0.21655295 0.25823138]
[0 1 2 4] [0.         0.05216898 0.0847746  0.14006238]
[1.17536400e+03 3.83388288e-05]
[ 0  1  2  4  8 15] [ 0.         -0.02573529  0.06832108  0.13817402  0.40992647  0.66819853]
[0.15757217 0.41667062]
[ 0  1  2  4  8 15] [0.         0.05710989 0.0819152  0.1246034  0.17623305 0.13931353]
[0.78820075 0.15496176]


C:\Users\fo72qev\AppData\Local\mambaforge\envs\PFP2\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning:

Covariance of the parameters could not be estimated



[-2.07050093e-02  6.17939771e+01]
[ 0  1  2  4  8 15] [ 0.          0.00148423 -0.00111317 -0.0122449  -0.04526902 -0.04638219]
[8.74656058e+02 1.17870148e-05]
[ 0  1  2  4  8 15] [0.         0.00439078 0.00109769 0.00768386 0.08196121 0.16684962]
[2.48589748e+02 2.47277708e-05]
[ 0  1  2  4  8 15] [0.         0.00150319 0.00338219 0.03006389 0.04735062 0.09319805]
[1.81618411e+02 9.72350844e-06]
[ 0  1  2  4  8 15] [ 0.         -0.01697531 -0.03317901 -0.02044753 -0.01003086  0.05285494]
[6.7709577e+02 1.7765331e-05]
[ 0  1  2  4  8 15] [ 0.         -0.00613497  0.00575153  0.03527607  0.10697853  0.18174847]
[2.42153996e+02 2.04167550e-05]
[0 1 2 4 8] [ 0.         -0.01390498  0.001545    0.00270375  0.05252993]
[0.0180994  0.25190296]
[0 1 2 4 8] [ 0.         -0.00263455  0.0052691   0.01844185  0.01317275]
[0.07944832 0.11114475]
[0 1 2 4 8] [0.         0.01410542 0.01299183 0.02783964 0.0471418 ]
[9.64543407e+01 1.82219561e-05]
[0 1 2 4 8] [ 0.         -0.00179986 -0.00071994  0.0

In [8]:
# get the maximum of the derivative
print(len(kinetic_curves))

330


In [22]:
ttpm1= 1, 1
ttpm2= 2, 1
ttpm4= 1, 2

xrange = np.arange(-0.5, 0.5, 0.1)
newfig = px.line()
for testparams in (ttpm1, ttpm2, ttpm4):
    newfig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams), mode="lines", opacity=1)
#    newfig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5)

newfig.show()

In [22]:
ttpm1= 1, 0, 1, 0
ttpm2= 2, 0, 1, 0
ttpm3= 1, 2, 1, -1
ttpm4= 1, 0, 2, 0

xrange = np.arange(-10, 10, 0.1)
newfig = px.line()
for testparams in (ttpm1, ttpm2, ttpm3, ttpm4):
    newfig.add_scatter(x=xrange, y=sigmoid(xrange, *testparams), mode="lines")
    newfig.add_scatter(x=xrange, y=sigmoid_derivative(xrange, *testparams), opacity=0.5)

newfig.show()